## **0. Download dataset**
**Note:** If you can't download using gdown due to limited number of downloads, please download it manually and upload it to your drive, then copy it from the drive to colab.
```python
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .
```

In [2]:
# https://drive.google.com/file/d/1aQ8OlUljwEm7RLZz8Qf4xzim0nDewHfU/view?usp=sharing
!gdown -q --id 1aQ8OlUljwEm7RLZz8Qf4xzim0nDewHfU

/home/banhmuy/miniconda3/envs/ds_env/lib/python3.11/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(


In [5]:
!unzip twitter_sentiment_analysis_3cls_dataset.zip
!mv Twitter_Data.csv ../data/

Archive:  twitter_sentiment_analysis_3cls_dataset.zip
  inflating: Twitter_Data.csv        


In [1]:
!pip -q install torch nltk

## **1. Import libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re  # Xử lý Biểu thức chính quy (Regular Expressions) cho tiền xử lý văn bản
import torch  # Thư viện PyTorch cơ sở
import torch.nn as nn  # Các module xây dựng mạng nơ-ron (như các lớp Linear, Conv)
import torch.optim as optim  # Các thuật toán tối ưu hóa (Optimizers) như Adam, SGD
import nltk  # Natural Language Toolkit (Bộ công cụ Xử lý Ngôn ngữ Tự nhiên)

nltk.download('stopwords')  # Tải xuống danh sách từ dừng (stopwords)

from sklearn.model_selection import train_test_split  # Chia dữ liệu thành tập huấn luyện và kiểm thử
from sklearn.feature_extraction.text import TfidfVectorizer  # Chuyển đổi văn bản thành vector TF-IDF
from nltk.corpus import stopwords  # Nhập danh sách từ dừng
from nltk.stem import SnowballStemmer  # Công cụ Stemming (đưa từ về dạng gốc)


/home/banhmuy/miniconda3/envs/ds_env/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/banhmuy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **2. Read dataset**

In [2]:
dataset_path = '../data/Twitter_Data.csv'
df = pd.read_csv(
    dataset_path
)
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [4]:
df.describe()

,category
count,162973.000000
mean,0.225436
std,0.781279
min,-1.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


## **3. Drop missing value**

In [5]:
null_rows = df.isnull().any(axis=1)
df[null_rows]

,clean_text,category
148,NaN,0.0
130448,the foundation stone northeast gas grid inaugu...,NaN
155642,dear terrorists you can run but you cant hide ...,NaN
155698,offense the best defence with mission shakti m...,NaN
155770,have always heard politicians backing out thei...,NaN
158693,modi government plans felicitate the faceless ...,NaN
158694,NaN,-1.0
159442,chidambaram gives praises modinomics,NaN
159443,NaN,0.0
160559,the reason why modi contested from seats 2014 ...,NaN


In [6]:
df = df.dropna()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 162969 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162969 non-null  object 
 1   category    162969 non-null  float64
dtypes: float64(1), object(1)
memory usage: 3.7+ MB


## **4. Preprocessing data**



### Regular Regression 
>> Mẫu `r'[^\w\s]'`

Phần quan trọng nhất là biểu thức chính quy:

* **`[...]`**: Khớp với bất kỳ ký tự nào nằm trong dấu ngoặc vuông.
* **`^` (Dấu mũ)**: Đặt ngay sau dấu ngoặc vuông, nó có nghĩa là **phủ định**. Nó khớp với **bất kỳ ký tự nào KHÔNG** nằm trong tập hợp theo sau nó.
* **`\w`**: Khớp với **bất kỳ ký tự chữ (a-z, A-Z), số (0-9) hoặc dấu gạch dưới (_)**. (`\w` viết tắt cho *word character*).
* **`\s`**: Khớp với **bất kỳ ký tự khoảng trắng (space, tab, newline)**. (`\s` viết tắt cho *whitespace*).

Tóm lại, **`[^\w\s]`** có nghĩa là **"tìm kiếm bất kỳ ký tự nào KHÔNG phải là chữ cái, số, dấu gạch dưới, hoặc khoảng trắng."** (tức là tất cả các dấu câu và ký tự đặc biệt).


>> Ví dụ Minh họa:

| Đầu vào (text) | Ký tự bị khớp (`[^\w\s]`) | Đầu ra |
| :--- | :--- | :--- |
| `"Hello! This is 100$. "` | `!`, `$`, `.` | `"Hello This is 100 "` |
| `"data-analysis@2025"` | `-`, `@` | `"dataanalysis2025"` |

Mẫu `r'https?:\/\/.*[\r\n]*'` được xây dựng để khớp với hầu hết các định dạng liên kết:

| Ký hiệu Regex | Ý nghĩa | Giải thích |
| :--- | :--- | :--- |
| **`http`** | Khớp chính xác với chuỗi ký tự "http". | Bắt đầu của URL. |
| **`s?`** | Khớp với ký tự "s" **một lần hoặc không lần nào**. | Điều này giúp khớp với cả `http` và `https`. |
| **`:\/`** | Khớp chính xác với chuỗi `:/`. | Kết hợp để khớp với `:` và hai dấu gạch chéo `//` (dấu `/` cần được thoát `\` vì nó có ý nghĩa đặc biệt trong một số ngữ cảnh regex). |
| **`.*`** | Khớp với **bất kỳ ký tự nào** (`.`) lặp lại **không hoặc nhiều lần** (`*`). | Khớp với phần còn lại của liên kết (ví dụ: `google.com/path?query=123`). Đây là phần mở rộng nhất. |
| **`[\r\n]*`** | Khớp với **ký tự xuống dòng** (`\r` hoặc `\n`) lặp lại **không hoặc nhiều lần**. | Đảm bảo rằng nếu liên kết nằm ở cuối dòng, ký tự xuống dòng đi kèm cũng được xóa sạch, ngăn không cho khoảng trắng thừa xuất hiện. |


In [8]:
def text_normalize(text):
    # Lowercasing
    text = text.lower()

    # Regular Expressions
    # Loại bỏ chữ viết tắt cũ của Retweet "RT" ở đầu dòng (thường gặp trên Twitter)
    text = re.sub(r'^rt[\s]+', '', text)

    # Hyperlinks removal 'http://' hoặc 'https://'
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

    # Punctuation removal (chỉ giữ lại từ, số, và khoảng trắng)
    text = re.sub(r'[^\w\s]', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    text = ' '.join(words)

    # Rút gọn từ về dạng gốc (Stemming)
    stemmer = SnowballStemmer('english')
    words = text.split()
    # Áp dụng stemming cho từng từ
    words = [stemmer.stem(word) for word in words]
    text = ' '.join(words)

    return text

In [9]:
text = """We love this! Would you go?
#talk #makememories #unplug
#relax #iphone #smartphone #wifi #connect...
http://fb.me/6N3LsUpCu
"""
text = text_normalize(text)
text

'love would go talk makememori unplug relax iphon smartphon wifi connect'

In [10]:
# Áp dụng hàm 'text_normalize' lên từng phần tử (từng chuỗi văn bản) trong cột 'clean_text'.
# Mục đích là làm sạch, chuẩn hóa, và tiền xử lý toàn bộ dữ liệu văn bản.
df['clean_text'] = df['clean_text'].apply(
    lambda x: text_normalize(x)
)

/tmp/ipykernel_4432/887339454.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['clean_text'].apply(


In [11]:
df

,clean_text,category
0,modi promis minimum govern maximum govern expe...,-1.0
1,talk nonsens continu drama vote modi,0.0
2,say vote modi welcom bjp told rahul main campa...,1.0
3,ask support prefix chowkidar name modi great s...,1.0
4,answer among power world leader today trump pu...,1.0
...,...,...
162975,456 crore paid neerav modi recov congress lead...,-1.0
162976,dear rss terrorist payal gawar modi kill 1000 ...,-1.0
162977,cover interact forum left,0.0
162978,big project came india modi dream project happ...,0.0


In [ ]:
vectorizer = TfidfVectorizer(
    # Max features (từ vựng) tối đa được sử dụng là 2000
    # TfidfVectorizer sẽ chọn 2000 từ có trọng số TF-IDF cao nhất
    max_features=2000
)
# 1. Học từ vựng và tính toán trọng số TF-IDF:
# fit_transform() thực hiện cả hai bước:
# a) Học từ điển (từ vựng) từ cột 'clean_text'
# b) Chuyển đổi văn bản thành ma trận TF-IDF (thường là ma trận thưa - sparse matrix)
# 2. Chuyển ma trận thưa sang ma trận dày (dense array) để dễ dàng thao tác với PyTorch hoặc các thư viện khác
X = vectorizer.fit_transform(
    df['clean_text']
).toarray()

In [ ]:
print(X.shape)

## **5. One-hot encoding label**

In [ ]:
n_classes = df['category'].nunique()
n_samples = df['category'].size

y = df['category'].to_numpy() + 1
y = y.astype(np.uint8)
y_encoded = np.array(
    [np.zeros(n_classes) for _ in range(n_samples)]
)
y_encoded[np.arange(n_samples), y] = 1

## **6. Create train, val, test set**

In [ ]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y_encoded, dtype=torch.float32)

In [ ]:
val_size = 0.2
test_size = 0.125
RANDOM_STATE = 2
is_shuffle = True

## **7. Define Softmax Regression model**

## **8. Training**

## **9. Evaluation**